In [1]:
!sudo apt update && sudo apt install -y ffmpeg
!sudo apt-get install -y cmake build-essential pkg-config libgoogle-perftools-dev
!python -m pip install --upgrade pip
!python -m pip install -U openai-whisper
!python -m pip install setuptools-rust transformers sentencepiece
!python -m pip install gradio
!pytohn -m pip install torch==1.12.0+cu116 torchvision==0.13.0+cu116 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu116

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,604 kB]
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main amd64 Packa

In [2]:
import gradio as gr
import whisper, torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_size = 'medium' # ["tiny", "base", "small", "medium", "large"]
model = whisper.load_model(model_size)

# Load pre-trained model and tokenizer
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
summ_model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
summ_model.to(device)


100%|█████████████████████████████████████| 1.42G/1.42G [00:45<00:00, 33.8MiB/s]


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [3]:
def summarize_paragraph(paragraph):
    inputs = tokenizer.encode_plus(paragraph, return_tensors='pt', max_length=512, truncation=True, padding='longest')
    inputs = inputs.to(device)
    # Generate the summary
    summary_ids = summ_model.generate(inputs.input_ids,
                                num_beams=4,
                                max_length=150,
                                early_stopping=True)
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    return summary

def speech_to_text(tmp_filename, audio_file):
    # print(tmp_filename)
    # print(audio_file)
    result_list = [None, None]
    summ_result_list = [None, None]
    for index, item in enumerate([tmp_filename, audio_file]):
        if item:
            result = model.transcribe(item, language="en")
            result_list[index] = result['text']

            summary = summarize_paragraph(result['text'])
            summ_result_list[index] = summary

    return result_list[0], summ_result_list[0], result_list[1], summ_result_list[1]


gr.Interface(
    title="Speech-to-Text Transcriber and Summarization with Deep Learning",
    fn=speech_to_text,
    inputs=[
        gr.Audio(source="microphone", type="filepath", label="MicroPhone Recorded Audio"),
        gr.Audio(source="upload", type="filepath", label="Upload Audio File"),
        ],
        outputs=[
            gr.inputs.Textbox(label="MicroPhone Recorded Audio file, transcribe Output Text"),
            gr.inputs.Textbox(label="MicroPhone Recorded Audio file, summary Output Text"),
            gr.inputs.Textbox(label="Upload Audio File, transcribe Output Text"),
            gr.inputs.Textbox(label="Upload Audio File, summary Output Text")
        ]
    ).launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://10c0b3d539cd75ce78.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
